In [ ]:
import pandas as pd

# Load NLP dataset
data = pd.read_csv('NLP_POS_sentences.csv')
data['id']=data.index
data.head()

,text,id
0,कथित Delhi Liquor Scam पर ED और Modi Govt झूठी...,0
1,Arvind Kejriwal ने किया Premium Bus Service Sc...,1
2,Sanjay Singh ने क्यों कहा कि PM Modi को नाक रग...,2
3,Kejriwal Govt के ₹45 Crore के CM House पर क्या...,3
4,CM Arvind Kejriwal और CM Bhagwant Mann का Adam...,4


In [ ]:
team_data = data[(data['id'] >= 520) & (data['id'] <= 539)]
team_data.head()
file_name="team_27_NLP_data.csv"
team_data.to_csv(file_name, index=False, encoding="utf-8")
print(f"Data saved to {file_name}")

Data saved to team_27_NLP_data.csv


In [ ]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 43.9 MB/s eta 0:00:00


In [ ]:
import stanza
import json

# Download and load the Hindi model from Stanza
stanza.download('hi')
nlp = stanza.Pipeline('hi')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: hi (Hindi) ...
INFO:stanza:File exists: /root/stanza_resources/hi/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: hi (Hindi):
| Processor | Package       |
-----------------------------
| tokenize  | hdtb          |
| pos       | hdtb_charlm   |
| lemma     | hdtb_nocharlm |
| depparse  | hdtb_charlm   |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
# List of sentences to annotate (replace this with your actual `team_data['text'].tolist()`)
sentences = team_data['text'].tolist()  # Assuming team_data is a DataFrame with a 'text' column

# POS tags mapping (from Stanza's UPOS tags to your desired tags)
pos_tag_map = {
    "NOUN": "NOUN",
    "PROPN": "PROPN",
    "VERB": "VERB",
    "ADJ": "ADJ",
    "ADV": "ADV",
    "ADP": "ADP",
    "PRON": "PRON",
    "DET": "DET",
    "CCONJ": "CONJ",
    "PART": "PART",
    "SCONJ": "CONJ",
    "NUM": "NUM",
    "PUNCT": "X",
    "SYM": "X",
    "X": "X",
    "AUX": "VERB"  # Map auxiliary verbs to VERB if needed
}

# Annotate each sentence
annotated_sentences = []
for idx, sentence in enumerate(sentences, start=1):
    # Process the sentence with Stanza
    doc = nlp(sentence)

    # List to hold POS tags for the sentence
    pos_tags = []

    # Extract POS tags from Stanza and map them
    for stanza_sentence in doc.sentences:
        for word in stanza_sentence.words:
            # Map the POS tag from Stanza to your required format
            pos_tags.append({
                "word": word.text,
                "entity": pos_tag_map.get(word.upos, "X")  # Default to "X" if tag not in map
            })

    # Add the annotated sentence to the list
    annotated_sentences.append({
        "sentence_id": idx + 519,  # Start IDs from 520 as per your requirement
        "sentence": sentence,
        "pos_tags": pos_tags
    })

# Save the annotated data to a JSON file
output_file = "annotated_pos_tags_stanza.json"
with open(output_file, "w", encoding="utf-8") as f:
    # Convert the annotated data to a serializable format
    json.dump(annotated_sentences, f, ensure_ascii=False, indent=4)

print(f"Annotated data saved to {output_file}")


Annotated data saved to annotated_pos_tags_stanza.json


In [ ]:
import json

# Path to your JSON file
json_file_path = "annotated_pos_tags_stanza.json"

# Load and print the JSON file content
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(json.dumps(data, ensure_ascii=False, indent=4))


[
    {
        "sentence_id": 520,
        "sentence": "मेरे प्यारे देशवासियो, भारत में Fit India Movement से तो आप परिचित हो ही गए होंगे।",
        "pos_tags": [
            {
                "word": "मेरे",
                "entity": "PRON"
            },
            {
                "word": "प्यारे",
                "entity": "ADJ"
            },
            {
                "word": "देशवासियो",
                "entity": "NOUN"
            },
            {
                "word": ",",
                "entity": "X"
            },
            {
                "word": "भारत",
                "entity": "PROPN"
            },
            {
                "word": "में",
                "entity": "ADP"
            },
            {
                "word": "Fit",
                "entity": "PROPN"
            },
            {
                "word": "India",
                "entity": "PROPN"
            },
            {
                "word": "Movement",
                "entity": "PROPN"

In [ ]:
import pandas as pd

# Load JSON data into a Pandas DataFrame
df = pd.json_normalize(data, record_path=['pos_tags'], meta=['sentence_id', 'sentence'])

# Display the first few rows of the DataFrame
df


,word,entity,sentence_id,sentence
0,मेरे,PRON,520,"मेरे प्यारे देशवासियो, भारत में Fit India Move..."
1,प्यारे,ADJ,520,"मेरे प्यारे देशवासियो, भारत में Fit India Move..."
2,देशवासियो,NOUN,520,"मेरे प्यारे देशवासियो, भारत में Fit India Move..."
3,",",X,520,"मेरे प्यारे देशवासियो, भारत में Fit India Move..."
4,भारत,PROPN,520,"मेरे प्यारे देशवासियो, भारत में Fit India Move..."
...,...,...,...,...
322,राम,PROPN,539,Ayodhya Ram Mandir Pran Pratishtha: दैनिक भास्...
323,मंदिर,PROPN,539,Ayodhya Ram Mandir Pran Pratishtha: दैनिक भास्...
324,का,ADP,539,Ayodhya Ram Mandir Pran Pratishtha: दैनिक भास्...
325,वर्चुअल,NOUN,539,Ayodhya Ram Mandir Pran Pratishtha: दैनिक भास्...


In [ ]:
import os
import pandas as pd

# Create a list of all image paths
image_folder = 'images/'  # Folder containing your images
image_paths = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.jpg')]

# Initialize an empty list to store the data
data = []

# Manually label each image
for image_path in image_paths:
    label = input(f"Does the image {image_path} contain a truck? (1 for Trucks, 0 for No Trucks): ")
    data.append([image_path, int(label)])

# Save the labeled data to a CSV file
df = pd.DataFrame(data, columns=["image_path", "label"])
df.to_csv("truck_annotations.csv", index=False)

print("Annotation complete! The data is saved to 'truck_annotations.csv'.")


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Dataset and DataLoader setup
class TruckDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

# Assuming you've annotated your images with a CSV
# Replace the path with your actual CSV file
csv_path = 'trucks_no_trucks_annotations.csv'
df = pd.read_csv(csv_path)

# Splitting the dataset into training and validation
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'])

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets and dataloaders
train_dataset = TruckDataset(train_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TruckDataset(val_df, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Step 3: Load pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Freeze all layers except the last fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to suit binary classification
model.fc = nn.Linear(model.fc.in_features, 2)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 4: Set up the loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Only the final layer parameters are being optimized (since all others are frozen)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Step 5: Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

# Step 6: Validate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = correct / total
print(f"Validation Accuracy: {val_acc:.4f}")

# Step 7: Save the model
torch.save(model.state_dict(), 'truck_classifier.pth')


In [ ]:
# Load the trained model
model.load_state_dict(torch.load('truck_classifier.pth'))
model.eval()

# Example image for prediction
image_path = "/content/images/your_image.jpg"
image = Image.open(image_path)
image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

# Prediction
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output, 1)

# Print prediction
if predicted.item() == 0:
    print("No Trucks")
else:
    print("Trucks")


TASK 3

In [ ]:
data=pd.read_csv("NLP.csv")
data

,annotation_id,annotator,created_at,id,label,lead_time,text,updated_at
0,1,1,2025-01-24T07:56:04.150522Z,520,"[{""start"":5,""end"":11,""text"":""प्यारे"",""labels"":...",784.351,"मेरे प्यारे देशवासियो, भारत में Fit India Move...",2025-01-24T08:28:02.979568Z
1,2,1,2025-01-24T07:56:09.819727Z,521,"[{""start"":0,""end"":2,""text"":""और"",""labels"":[""\""C...",470.674,"और श्वेता का सुझाव सही है कि मुझे, इसको, जनवरी...",2025-01-24T08:29:12.831905Z
2,3,1,2025-01-24T08:06:01.007955Z,522,"[{""start"":5,""end"":8,""text"":""पता"",""labels"":[""\""...",138.984,आपको पता होगा कि भारत के ही प्रयास से पिछले सा...,2025-01-24T08:29:47.135298Z
3,4,1,2025-01-24T08:08:44.505640Z,523,"[{""start"":0,""end"":5,""text"":""Bihar"",""labels"":[""...",163.292,Bihar Politics: राजस्थान-एमपी की तर्ज पर मंत्र...,2025-01-24T08:08:44.505660Z
4,5,1,2025-01-24T08:09:52.883134Z,524,"[{""start"":0,""end"":6,""text"":""Bharat"",""labels"":[...",68.199,Bharat Nyay Jodo Yatra : Rahul Gandhi ने नीतीश...,2025-01-24T08:09:52.883151Z
5,6,1,2025-01-24T08:12:22.175206Z,525,"[{""start"":0,""end"":5,""text"":""Bihar"",""labels"":[""...",158.984,Bihar Politics: बिहार NDA सरकार की पहली कैबिने...,2025-01-24T08:17:44.663581Z
6,7,1,2025-01-24T08:15:27.729359Z,526,"[{""start"":0,""end"":5,""text"":""Bihar"",""labels"":[""...",216.998,Bihar Political Crises: नीतीश कुमार 9वीं बार ब...,2025-01-24T08:16:22.719040Z
7,8,1,2025-01-24T08:17:31.447490Z,527,"[{""start"":0,""end"":6,""text"":""Bharat"",""labels"":[...",65.567,Bharat Nyay Jodo Yatra: राहुल गांधी से मिल सकत...,2025-01-24T08:17:31.447511Z
8,9,1,2025-01-24T08:20:44.484492Z,528,"[{""start"":0,""end"":6,""text"":""Nitish"",""labels"":[...",174.418,Nitish Kumar Resigns: नीतीश कुमार ने दिया इस्त...,2025-01-24T08:20:44.484511Z
9,10,1,2025-01-24T08:21:54.956540Z,529,"[{""start"":0,""end"":3,""text"":""Lok"",""labels"":[""\""...",70.245,Lok Sabha Election 2024: Uttar Pradesh में SPA...,2025-01-24T08:21:54.956557Z
